In [ ]:
#패키지 설치
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from thundersvm import SVC
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
train = pd.read_csv("train.csv")

# 1차 변수 제거 및 타입 변경

In [ ]:
train = train.replace({'Y':1,'N':0}) # 데이터 셋에 있는 Y, N을 1,0으로 변경

In [ ]:
train['SIGHT_LEFT_CATE'] =train['SIGHT_LEFT'] #  좌,우 시력 범주화 위해 임의로 생성
train['SIGHT_RIGHT_CATE']= train['SIGHT_RIGHT']

In [ ]:
train.loc[train['SIGHT_LEFT'] <= 3,'SIGHT_LEFT_CATE']='0' # 실명이 아니면 0 실명이면 1
train.loc[train['SIGHT_LEFT'] >3, 'SIGHT_LEFT_CATE']='1'
train.loc[train['SIGHT_RIGHT'] <= 3,'SIGHT_RIGHT_CATE']='0'
train.loc[train['SIGHT_RIGHT'] >3, 'SIGHT_RIGHT_CATE']='1'

In [ ]:
#변수 카테고리화
train= train.astype({'SEX':'category','HEAR_LEFT':'category', 'SIDO': 'category', 'HEAR_RIGHT':'category', 'SMK_STAT': 'category'})
train= train.astype({'AGE':'category','HEIGHT':'category', 'WEIGHT': 'category'})
train= train.astype({'OLIG_PROTE':'category', 'CRS_YN': 'category'})
train= train.astype({'DRK_YN':'category', 'HCHK_OE_INSPEC_YN':'category', 'TTR_YN':'category'})

In [ ]:
# 단일 값, 흡연 판별에 불필요한 속성 제거 / ID, SIDO, 구강검진 유무, 치석 유무 / 청각(좌,우)
train = train.drop(train.columns[[0,6,3,9,10,25,27]],axis=1)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   SEX               100000 non-null  category
 1   AGE               100000 non-null  category
 2   HEIGHT            100000 non-null  category
 3   WEIGHT            100000 non-null  category
 4   SIGHT_LEFT        99979 non-null   float64 
 5   SIGHT_RIGHT       99979 non-null   float64 
 6   BP_HIGH           99425 non-null   float64 
 7   BP_LWST           99425 non-null   float64 
 8   BLDS              99409 non-null   float64 
 9   TOT_CHOLE         33340 non-null   float64 
 10  TRIGLYCERIDE      33340 non-null   float64 
 11  HDL_CHOLE         33339 non-null   float64 
 12  LDL_CHOLE         32707 non-null   float64 
 13  HMG               99410 non-null   float64 
 14  OLIG_PROTE        98973 non-null   category
 15  CREATININE        99409 non-null   float64 
 16  SGO

In [ ]:
#데이터의 50% 이상이 결측치인 변수 제거 / 총 콜레스테롤, 트리글리세라이드, HDL콜레스테롤, LDL콜레스테롤, 치아우식증 유무
train = train.drop(train.columns[[9,10,11,12,20]], axis=1)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   SEX               100000 non-null  category
 1   AGE               100000 non-null  category
 2   HEIGHT            100000 non-null  category
 3   WEIGHT            100000 non-null  category
 4   SIGHT_LEFT        99979 non-null   float64 
 5   SIGHT_RIGHT       99979 non-null   float64 
 6   BP_HIGH           99425 non-null   float64 
 7   BP_LWST           99425 non-null   float64 
 8   BLDS              99409 non-null   float64 
 9   HMG               99410 non-null   float64 
 10  OLIG_PROTE        98973 non-null   category
 11  CREATININE        99409 non-null   float64 
 12  SGOT_AST          99410 non-null   float64 
 13  SGOT_ALT          99410 non-null   float64 
 14  GAMMA_GTP         99410 non-null   float64 
 15  DRK_YN            100000 non-null  category
 16  SMK

# 시력 범주화

In [ ]:
#원래 있던 좌,우 시력 변수 삭제
train = train.drop(train.columns[[4,5]], axis=1)

In [ ]:
train= train.astype({'SIGHT_LEFT_CATE':'category','SIGHT_RIGHT_CATE':'category'})

In [ ]:
#성별 분리
women = train[train.SEX==2]
men = train[train.SEX==1]

# 결측값 대체

In [ ]:
#KNN 이용에 앞서 정규화 / 성별에 나눠서
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
men= pd.DataFrame(scaler.fit_transform(men), columns= men.columns)
women= pd.DataFrame(scaler.fit_transform(women), columns= women.columns)

In [ ]:
men.describe()

,SEX,AGE,HEIGHT,WEIGHT,BP_HIGH,BP_LWST,BLDS,HMG,OLIG_PROTE,CREATININE,SGOT_AST,SGOT_ALT,GAMMA_GTP,DRK_YN,SMK_STAT,SIGHT_LEFT_CATE,SIGHT_RIGHT_CATE
count,54646.0,54646.000000,54646.000000,54646.000000,54441.000000,54441.000000,54435.000000,54436.000000,54287.000000,54436.000000,54436.000000,54436.000000,54436.000000,54646.000000,54646.000000,54634.000000,54635.000000
mean,0.0,0.413161,0.522948,0.320618,0.338639,0.296693,0.124865,0.580837,0.021932,0.012010,0.004471,0.007986,0.047898,0.780624,0.517046,0.003496,0.003185
std,0.0,0.216952,0.146107,0.120271,0.102193,0.082756,0.045687,0.070625,0.094088,0.006645,0.006330,0.010332,0.061587,0.413827,0.400986,0.059024,0.056344
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,0.230769,0.444444,0.263158,0.270677,0.228814,0.102967,0.541667,0.000000,0.009602,0.003025,0.004437,0.020060,1.000000,0.000000,0.000000,0.000000
50%,0.0,0.384615,0.555556,0.315789,0.330827,0.296610,0.115183,0.583333,0.000000,0.012346,0.003866,0.006265,0.030090,1.000000,0.500000,0.000000,0.000000
75%,0.0,0.538462,0.666667,0.368421,0.398496,0.347458,0.132635,0.625000,0.000000,0.013717,0.004874,0.009136,0.053159,1.000000,1.000000,0.000000,0.000000
max,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# 인접이웃 3개를 기준으로 결측치 대체 / 성별에 나눠서
from sklearn.impute import KNNImputer
imputer =KNNImputer(n_neighbors=3)
men=pd.DataFrame(imputer.fit_transform(men), columns=men.columns)
women=pd.DataFrame(imputer.fit_transform(women), columns=women.columns)

In [ ]:
# 정규화된 데이터 역 정규화 / 성별에 나눠서
men= pd.DataFrame(scaler.inverse_transform(men), columns=men.columns)
women= pd.DataFrame(scaler.inverse_transform(women), columns=women.columns)

In [ ]:
#결측치 및 전체 데이터 중 비중 확인 /남자
missing_df = men.isnull().sum().reset_index()
missing_df.columns=['column','count']
missing_df['ratio']= missing_df['count'] / men.shape[0]
missing_df.loc[missing_df['ratio']!=0]

# 결측치 사라진 모습

,column,count,ratio


In [ ]:
#결측치 및 전체 데이터 중 비중 확인 /여자
missing_df = women.isnull().sum().reset_index()
missing_df.columns=['column','count']
missing_df['ratio']= missing_df['count'] / women.shape[0]
missing_df.loc[missing_df['ratio']!=0]

# 결측치 사라진 모습

,column,count,ratio


In [ ]:
#정규화를 했다가 다시 역정규화를 해서 상수형으로 됨
men.describe()

,SEX,AGE,HEIGHT,WEIGHT,BP_HIGH,BP_LWST,BLDS,HMG,OLIG_PROTE,CREATININE,SGOT_AST,SGOT_ALT,GAMMA_GTP,DRK_YN,SMK_STAT,SIGHT_LEFT_CATE,SIGHT_RIGHT_CATE
count,54646.0,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000
mean,2.0,46.855488,153.762123,57.252503,119.579042,79.451793,101.322681,13.235947,1.109853,1.239784,9.865177,16.982534,48.753769,0.780624,1.034092,0.003501,0.003184
std,0.0,14.101892,8.035903,10.223036,17.050584,10.993215,22.045958,1.037961,0.469555,0.629530,12.528698,20.659686,61.367527,0.413827,0.801973,0.059034,0.056339
min,2.0,20.000000,125.000000,30.000000,63.000000,40.000000,41.000000,4.700000,1.000000,0.100000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.0,35.000000,149.444444,52.368421,108.203008,70.432203,90.732984,12.662500,1.000000,1.011248,6.998992,9.888280,21.020060,1.000000,0.000000,0.000000,0.000000
50%,2.0,45.000000,155.555556,56.842105,118.248120,79.449153,96.633508,13.275000,1.000000,1.271605,8.665378,13.548160,31.030090,1.000000,1.000000,0.000000,0.000000
75%,2.0,55.000000,161.666667,61.315789,129.548872,86.211864,105.062827,13.887500,1.000000,1.401783,10.665042,19.299400,53.052156,1.000000,2.000000,0.000000,0.000000
max,2.0,85.000000,180.000000,115.000000,230.000000,173.000000,524.000000,19.400000,6.000000,95.000000,1984.000000,2004.000000,999.000000,1.000000,2.000000,1.000000,1.000000


In [ ]:
men['OLIG_PROTE_CATE'] = 0 # 요단백 범주화에 앞서 가변수 생성
women['OLIG_PROTE_CATE'] = 0 # 요단백 범주화에 앞서 가변수 생성

In [ ]:
men['SIGHT_RIGHT_CATE'].value_counts()


0.0    54472
1.0      174
Name: SIGHT_RIGHT_CATE, dtype: int64

In [ ]:
#0.5 기준으로 새롭게 변경 / 남자
men.loc[(men['OLIG_PROTE'] >0) & (men['OLIG_PROTE'] <=1.5),'OLIG_PROTE_CATE']= '1'
men.loc[(men['OLIG_PROTE'] >1.5) & (men['OLIG_PROTE'] <=2), 'OLIG_PROTE_CATE']='2'
men.loc[(men['OLIG_PROTE'] >2) & (men['OLIG_PROTE'] <=2.5),'OLIG_PROTE_CATE']='2'
men.loc[(men['OLIG_PROTE'] >2.5) & (men['OLIG_PROTE'] <=3), 'OLIG_PROTE_CATE']='3'
men.loc[(men['OLIG_PROTE']>3) & (men['OLIG_PROTE'] <=3.5),'OLIG_PROTE_CATE']='3'
men.loc[(men['OLIG_PROTE'] >3.5) & (men['OLIG_PROTE'] <=4), 'OLIG_PROTE_CATE']='4'
men.loc[(men['OLIG_PROTE'] >4) & (men['OLIG_PROTE'] <=4.5),'OLIG_PROTE_CATE']='4'
men.loc[(men['OLIG_PROTE'] >4.5) & (men['OLIG_PROTE'] <=5), 'OLIG_PROTE_CATE']='5'
men.loc[(men['OLIG_PROTE'] >5) & (men['OLIG_PROTE'] <=5.5),'OLIG_PROTE_CATE']='5'
men.loc[(men['OLIG_PROTE'] >5.5) & (men['OLIG_PROTE'] <=6.5), 'OLIG_PROTE_CATE']='6'
men.loc[(men['SIGHT_LEFT_CATE'] >0) & (men['SIGHT_LEFT_CATE'] <=0.5),'SIGHT_LEFT_CATE']='0'

In [ ]:
women.loc[(women['OLIG_PROTE'] >0) & (women['OLIG_PROTE'] <=1.5),'OLIG_PROTE_CATE']='1'
women.loc[(women['OLIG_PROTE'] >1.5) & (women['OLIG_PROTE'] <=2), 'OLIG_PROTE_CATE']='2'
women.loc[(women['OLIG_PROTE'] >2) & (women['OLIG_PROTE'] <=2.5),'OLIG_PROTE_CATE']='2'
women.loc[(women['OLIG_PROTE'] >2.5) & (women['OLIG_PROTE'] <=3), 'OLIG_PROTE_CATE']='3'
women.loc[(women['OLIG_PROTE']>3) & (women['OLIG_PROTE'] <=3.5),'OLIG_PROTE_CATE']='3'
women.loc[(women['OLIG_PROTE'] >3.5) & (women['OLIG_PROTE'] <=4), 'OLIG_PROTE_CATE']='4'
women.loc[(women['OLIG_PROTE'] >4) & (women['OLIG_PROTE'] <=4.5),'OLIG_PROTE_CATE']='4'
women.loc[(women['OLIG_PROTE'] >4.5) & (women['OLIG_PROTE'] <=5), 'OLIG_PROTE_CATE']='5'
women.loc[(women['OLIG_PROTE'] >5) & (women['OLIG_PROTE'] <=5.5),'OLIG_PROTE_CATE']='5'
women.loc[(women['OLIG_PROTE'] >5.5) & (women['OLIG_PROTE'] <=6.5), 'OLIG_PROTE_CATE']='6'
women.loc[(women['SIGHT_LEFT_CATE'] >0) & (women['SIGHT_LEFT_CATE'] <=0.5),'SIGHT_LEFT_CATE']='0'

In [ ]:
men.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54646 entries, 0 to 54645
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SEX               54646 non-null  float64
 1   AGE               54646 non-null  float64
 2   HEIGHT            54646 non-null  float64
 3   WEIGHT            54646 non-null  float64
 4   BP_HIGH           54646 non-null  float64
 5   BP_LWST           54646 non-null  float64
 6   BLDS              54646 non-null  float64
 7   HMG               54646 non-null  float64
 8   OLIG_PROTE        54646 non-null  float64
 9   CREATININE        54646 non-null  float64
 10  SGOT_AST          54646 non-null  float64
 11  SGOT_ALT          54646 non-null  float64
 12  GAMMA_GTP         54646 non-null  float64
 13  DRK_YN            54646 non-null  float64
 14  SMK_STAT          54646 non-null  float64
 15  SIGHT_LEFT_CATE   54646 non-null  object 
 16  SIGHT_RIGHT_CATE  54646 non-null  float6

In [ ]:
#원래 있던 요단백 변수 제거
men = men.drop(men.columns[8],axis=1)
women = women.drop(women.columns[8],axis=1)

In [ ]:
#새로 만들었던 가변수들 원래 변수명으로 변경
men.rename(columns = {'SIGHT_LEFT_CATE':'SIGHT_LEFT','SIGHT_RIGHT_CATE':'SIGHT_RIGHT','OLIG_PROTE_CATE':'OLIG_PROTE'},inplace=True)
women.rename(columns = {'SIGHT_LEFT_CATE':'SIGHT_LEFT','SIGHT_RIGHT_CATE':'SIGHT_RIGHT','OLIG_PROTE_CATE':'OLIG_PROTE'},inplace=True)

In [ ]:
# 소수점 제거를 위해 범주형 데이터 int로 변경
men= men.astype({'SEX':'int', 'SMK_STAT': 'int', 'HEIGHT': 'int', 'WEIGHT':'int'})
men= men.astype({'AGE':'int', 'DRK_YN' : 'int','SIGHT_LEFT': 'int', 'SIGHT_RIGHT': 'int'})

women= women.astype({'SEX':'int', 'SMK_STAT': 'int', 'HEIGHT': 'int', 'WEIGHT':'int'})
women= women.astype({'AGE':'int', 'DRK_YN' : 'int','SIGHT_LEFT': 'int', 'SIGHT_RIGHT': 'int'})

In [ ]:
# 범주형 데이터로 변환 남자
men= men.astype({'SEX':'category', 'SMK_STAT': 'category', 'HEIGHT': 'category', 'WEIGHT':'category'})
men= men.astype({'AGE':'category', 'DRK_YN': 'category', 'OLIG_PROTE' : 'category'})
men = men.astype({'SIGHT_LEFT': 'category', 'SIGHT_RIGHT': 'category'})

# 범주형 데이터로 변환 여자
women= women.astype({'SEX':'category', 'SMK_STAT': 'category', 'HEIGHT': 'category', 'WEIGHT':'category'})
women= women.astype({'AGE':'category', 'DRK_YN': 'category', 'OLIG_PROTE' : 'category'})
women = women.astype({'SIGHT_LEFT': 'category', 'SIGHT_RIGHT': 'category'})

In [ ]:
men.describe()


,BP_HIGH,BP_LWST,BLDS,HMG,CREATININE,SGOT_AST,SGOT_ALT,GAMMA_GTP
count,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000,54646.000000
mean,119.579042,79.451793,101.322681,13.235947,1.239784,9.865177,16.982534,48.753769
std,17.050584,10.993215,22.045958,1.037961,0.629530,12.528698,20.659686,61.367527
min,63.000000,40.000000,41.000000,4.700000,0.100000,1.000000,1.000000,1.000000
25%,108.203008,70.432203,90.732984,12.662500,1.011248,6.998992,9.888280,21.020060
50%,118.248120,79.449153,96.633508,13.275000,1.271605,8.665378,13.548160,31.030090
75%,129.548872,86.211864,105.062827,13.887500,1.401783,10.665042,19.299400,53.052156
max,230.000000,173.000000,524.000000,19.400000,95.000000,1984.000000,2004.000000,999.000000


# 이상치 제거

In [ ]:
#이상치를 결측치로
men.loc[men['HMG'] < 5, 'HMG']=np.NaN
men.loc[men['HMG'] >=20,'HMG']= np.NaN
men.loc[men['BP_HIGH'] <= 40,'BP_HIGH']=np.nan
men.loc[men['BP_HIGH'] >= 180,'BP_HIGH']=np.nan
men.loc[men['BP_LWST'] <= 40,'BP_LWST']=np.nan
men.loc[men['BP_LWST'] >= 180,'BP_LWST']=np.nan
men.loc[men['CREATININE'] >= 70, 'CREATININE']=np.nan
men.loc[men['SGOT_AST'] >= 10000, 'SGOT_AST']=np.nan
men.loc[men['SGOT_ALT'] >= 10000, 'SGOT_ALT']=np.nan
men.loc[men['SMK_STAT'] == 2, 'SMK_STAT']=np.nan
men.loc[(men['SEX']==1) & (men['GAMMA_GTP'] >= 550), 'GAMMA_GTP']=np.nan
men.loc[men['BLDS'] >= 300, 'BLDS']=np.NaN


#여자
women.loc[women['HMG'] < 5, 'HMG']=np.NaN
women.loc[women['HMG'] >=20,'HMG']= np.NaN
women.loc[women['BP_HIGH'] <= 40,'BP_HIGH']=np.nan
women.loc[women['BP_HIGH'] >= 180,'BP_HIGH']=np.nan
women.loc[women['BP_LWST'] <= 40,'BP_LWST']=np.nan
women.loc[women['BP_LWST'] >= 180,'BP_LWST']=np.nan
women.loc[women['CREATININE'] >= 70, 'CREATININE']=np.nan
women.loc[women['SGOT_AST'] >= 10000, 'SGOT_AST']=np.nan
women.loc[women['SGOT_ALT'] >= 10000, 'SGOT_ALT']=np.nan
women.loc[women['SMK_STAT'] == 2, 'SMK_STAT']=np.nan
women.loc[(women['SEX']==2) & (women['GAMMA_GTP'] >= 380), 'GAMMA_GTP']=np.nan
women.loc[women['BLDS'] >= 300, 'BLDS']=np.NaN


In [ ]:
#결측치 및 전체 데이터 중 비중 확인 여자
missing_df = women.isnull().sum().reset_index()
missing_df.columns=['column','count']
missing_df['ratio']= missing_df['count'] / women.shape[0]
missing_df.loc[missing_df['ratio']!=0]

,column,count,ratio
4,BP_HIGH,90,0.001984
5,BP_LWST,1,0.000022
6,BLDS,47,0.001036
7,HMG,1,0.000022
8,CREATININE,1,0.000022
11,GAMMA_GTP,46,0.001014
13,SMK_STAT,1340,0.029545


In [ ]:
#결측치 및 전체 데이터 중 비중 확인 남자
missing_df = men.isnull().sum().reset_index()
missing_df.columns=['column','count']
missing_df['ratio']= missing_df['count'] / men.shape[0]
missing_df.loc[missing_df['ratio']!=0]


,column,count,ratio
4,BP_HIGH,165,0.003019
5,BP_LWST,2,0.000037
6,BLDS,51,0.000933
7,HMG,2,0.000037
8,CREATININE,1,0.000018
13,SMK_STAT,18536,0.339201


다시 위에 결측치 KNN 실행

In [ ]:
men.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54646 entries, 0 to 54645
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   SEX          54646 non-null  category
 1   AGE          54646 non-null  category
 2   HEIGHT       54646 non-null  category
 3   WEIGHT       54646 non-null  category
 4   BP_HIGH      54481 non-null  float64 
 5   BP_LWST      54644 non-null  float64 
 6   BLDS         54595 non-null  float64 
 7   HMG          54644 non-null  float64 
 8   CREATININE   54645 non-null  float64 
 9   SGOT_AST     54646 non-null  float64 
 10  SGOT_ALT     54646 non-null  float64 
 11  GAMMA_GTP    54646 non-null  float64 
 12  DRK_YN       54646 non-null  category
 13  SMK_STAT     36110 non-null  category
 14  SIGHT_LEFT   54646 non-null  category
 15  SIGHT_RIGHT  54646 non-null  category
 16  OLIG_PROTE   54646 non-null  category
dtypes: category(9), float64(8)
memory usage: 3.8 MB


In [ ]:
train['OLIG_PROTE'].value_counts()

1.0    92977
2.0     3377
3.0     1759
4.0      620
5.0      187
6.0       53
Name: OLIG_PROTE, dtype: int64

In [ ]:
#KNN 이용에 앞서 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
men= pd.DataFrame(scaler.fit_transform(men), columns= men.columns)
women= pd.DataFrame(scaler.fit_transform(women), columns= women.columns)



In [ ]:
#KNN 이용에 앞서 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
men= pd.DataFrame(scaler.fit_transform(men), columns= men.columns)
women= pd.DataFrame(scaler.fit_transform(women), columns= women.columns)

# 인접이웃 3개를 기준으로 결측치 대체
from sklearn.impute import KNNImputer
imputer =KNNImputer(n_neighbors=3)
men=pd.DataFrame(imputer.fit_transform(men), columns=men.columns)
women=pd.DataFrame(imputer.fit_transform(women), columns=women.columns)

# 정규화된 데이터 역 정규화
men= pd.DataFrame(scaler.inverse_transform(men), columns=men.columns)
women= pd.DataFrame(scaler.inverse_transform(women), columns=women.columns)


men= men.astype({'SMK_STAT': 'float'})
women= women.astype({'SMK_STAT': 'float'})


men.loc[(men['SMK_STAT'] >0) & (men['SMK_STAT'] <=0.5),'SMK_STAT']='0'
men.loc[(men['SMK_STAT'] >0.5) & (men['SMK_STAT'] <=1.5), 'SMK_STAT']='1'
women.loc[(women['SMK_STAT'] >0) & (women['SMK_STAT'] <=0.5),'SMK_STAT']='0'
women.loc[(women['SMK_STAT'] >0.5) & (women['SMK_STAT'] <=1.5), 'SMK_STAT']='1'

# 소수점 제거를 위해 범주형 데이터 int로 변경
men= men.astype({'SEX':'int', 'SMK_STAT': 'int', 'HEIGHT': 'int', 'WEIGHT':'int'})
men= men.astype({'AGE':'int', 'DRK_YN' : 'int','SIGHT_LEFT': 'int', 'SIGHT_RIGHT': 'int'})


# 소수점 제거를 위해 범주형 데이터 int로 변경
women= women.astype({'SEX':'int', 'SMK_STAT': 'int', 'HEIGHT': 'int', 'WEIGHT':'int'})
women= women.astype({'AGE':'int', 'DRK_YN' : 'int','SIGHT_LEFT':'int'})



# 범주형 데이터로 변환
men= men.astype({'SEX':'category', 'SMK_STAT': 'category', 'HEIGHT': 'category', 'WEIGHT':'category'})
men= men.astype({'AGE':'category', 'DRK_YN': 'category', 'OLIG_PROTE' : 'category'})
men =men.astype({'SIGHT_LEFT': 'category', 'SIGHT_RIGHT': 'category'})


# 범주형 데이터로 변환
women= women.astype({'SEX':'category', 'SMK_STAT': 'category', 'HEIGHT': 'category', 'WEIGHT':'category'})
women= women.astype({'AGE':'category', 'DRK_YN': 'category', 'OLIG_PROTE' : 'category'})
women =women.astype({'SIGHT_LEFT': 'category', 'SIGHT_RIGHT': 'category'})

In [ ]:
# 인접이웃 3개를 기준으로 결측치 대체
from sklearn.impute import KNNImputer
imputer =KNNImputer(n_neighbors=3)
men=pd.DataFrame(imputer.fit_transform(men), columns=men.columns)
women=pd.DataFrame(imputer.fit_transform(women), columns=women.columns)

In [ ]:
# 정규화된 데이터 역 정규화
men= pd.DataFrame(scaler.inverse_transform(men), columns=men.columns)
women= pd.DataFrame(scaler.inverse_transform(women), columns=women.columns)

In [ ]:
#결측치 및 전체 데이터 중 비중 확인 /남자
missing_df = men.isnull().sum().reset_index()
missing_df.columns=['column','count']
missing_df['ratio']= missing_df['count'] / men.shape[0]
missing_df.loc[missing_df['ratio']!=0]

# 결측치 사라진 모습

,column,count,ratio


In [ ]:
#결측치 및 전체 데이터 중 비중 확인 /여자
missing_df = women.isnull().sum().reset_index()
missing_df.columns=['column','count']
missing_df['ratio']= missing_df['count'] / women.shape[0]
missing_df.loc[missing_df['ratio']!=0]

# 결측치 사라진 모습

,column,count,ratio


In [ ]:
men['OLIG_PROTE'].value_counts()

1.0    51027
2.0     1994
3.0     1070
4.0      397
5.0      121
6.0       37
Name: OLIG_PROTE, dtype: int64

In [ ]:
men['SMK_STAT'].value_counts()

1      29242
0.0    25404
Name: SMK_STAT, dtype: int64

In [ ]:
men= men.astype({'SMK_STAT': 'float'})
women= women.astype({'SMK_STAT': 'float'})

In [ ]:
men.loc[(men['SMK_STAT'] >0) & (men['SMK_STAT'] <=0.5),'SMK_STAT']='0'
men.loc[(men['SMK_STAT'] >0.5) & (men['SMK_STAT'] <=1.5), 'SMK_STAT']='1'
women.loc[(women['SMK_STAT'] >0) & (women['SMK_STAT'] <=0.5),'SMK_STAT']='0'
women.loc[(women['SMK_STAT'] >0.5) & (women['SMK_STAT'] <=1.5), 'SMK_STAT']='1'

In [ ]:
men.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54646 entries, 0 to 54645
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SEX          54646 non-null  float64
 1   AGE          54646 non-null  float64
 2   HEIGHT       54646 non-null  float64
 3   WEIGHT       54646 non-null  float64
 4   BP_HIGH      54646 non-null  float64
 5   BP_LWST      54646 non-null  float64
 6   BLDS         54646 non-null  float64
 7   HMG          54646 non-null  float64
 8   CREATININE   54646 non-null  float64
 9   SGOT_AST     54646 non-null  float64
 10  SGOT_ALT     54646 non-null  float64
 11  GAMMA_GTP    54646 non-null  float64
 12  DRK_YN       54646 non-null  float64
 13  SMK_STAT     54646 non-null  object 
 14  SIGHT_LEFT   54646 non-null  float64
 15  SIGHT_RIGHT  54646 non-null  float64
 16  OLIG_PROTE   54646 non-null  float64
dtypes: float64(16), object(1)
memory usage: 7.1+ MB


In [ ]:
women.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45354 entries, 0 to 45353
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SEX          45354 non-null  float64
 1   AGE          45354 non-null  float64
 2   HEIGHT       45354 non-null  float64
 3   WEIGHT       45354 non-null  float64
 4   BP_HIGH      45354 non-null  float64
 5   BP_LWST      45354 non-null  float64
 6   BLDS         45354 non-null  float64
 7   HMG          45354 non-null  float64
 8   CREATININE   45354 non-null  float64
 9   SGOT_AST     45354 non-null  float64
 10  SGOT_ALT     45354 non-null  float64
 11  GAMMA_GTP    45354 non-null  float64
 12  DRK_YN       45354 non-null  float64
 13  SMK_STAT     45354 non-null  object 
 14  SIGHT_LEFT   45354 non-null  float64
 15  SIGHT_RIGHT  45354 non-null  float64
 16  OLIG_PROTE   45354 non-null  float64
dtypes: float64(16), object(1)
memory usage: 5.9+ MB


In [ ]:
# 소수점 제거를 위해 범주형 데이터 int로 변경
men= men.astype({'SEX':'int', 'SMK_STAT': 'int', 'HEIGHT': 'int', 'WEIGHT':'int'})
men= men.astype({'AGE':'int', 'DRK_YN' : 'int','SIGHT_LEFT': 'int', 'SIGHT_RIGHT': 'int'})


# 소수점 제거를 위해 범주형 데이터 int로 변경
women= women.astype({'SEX':'int', 'SMK_STAT': 'int', 'HEIGHT': 'int', 'WEIGHT':'int'})
women= women.astype({'AGE':'int', 'DRK_YN' : 'int','SIGHT_LEFT': 'int', 'SIGHT_RIGHT': 'int'})

In [ ]:
# 범주형 데이터로 변환
men= men.astype({'SEX':'category', 'SMK_STAT': 'category', 'HEIGHT': 'category', 'WEIGHT':'category'})
men= men.astype({'AGE':'category', 'DRK_YN': 'category', 'OLIG_PROTE' : 'category'})
men =men.astype({'SIGHT_LEFT': 'category', 'SIGHT_RIGHT': 'category'})


# 범주형 데이터로 변환
women= women.astype({'SEX':'category', 'SMK_STAT': 'category', 'HEIGHT': 'category', 'WEIGHT':'category'})
women= women.astype({'AGE':'category', 'DRK_YN': 'category', 'OLIG_PROTE' : 'category'})
women =women.astype({'SIGHT_LEFT': 'category', 'SIGHT_RIGHT': 'category'})

In [ ]:
men.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54646 entries, 0 to 54645
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   SEX          54646 non-null  category
 1   AGE          54646 non-null  category
 2   HEIGHT       54646 non-null  category
 3   WEIGHT       54646 non-null  category
 4   BP_HIGH      54646 non-null  float64 
 5   BP_LWST      54646 non-null  float64 
 6   BLDS         54646 non-null  float64 
 7   HMG          54646 non-null  float64 
 8   CREATININE   54646 non-null  float64 
 9   SGOT_AST     54646 non-null  float64 
 10  SGOT_ALT     54646 non-null  float64 
 11  GAMMA_GTP    54646 non-null  float64 
 12  DRK_YN       54646 non-null  category
 13  SMK_STAT     54646 non-null  category
 14  SIGHT_LEFT   54646 non-null  category
 15  SIGHT_RIGHT  54646 non-null  category
 16  OLIG_PROTE   54646 non-null  category
dtypes: category(9), float64(8)
memory usage: 3.8 MB


# train데이터 train/val/test .7/.1/.2 로 분리

In [ ]:
#x,y 분리
men_y= men['SMK_STAT']
men_x= men.drop(['SMK_STAT'], axis=1)
#x,y 분리
women_y= women['SMK_STAT']
women_x= women.drop(['SMK_STAT'], axis=1)

In [ ]:
x_train_women, x_test_women, y_train_women, y_test_women = train_test_split(women_x,women_y,test_size= 0.3, shuffle=True, random_state= 1004)

In [ ]:
x_train_men, x_test_men, y_train_men, y_test_men = train_test_split(men_x,men_y,test_size= 0.3, shuffle=True, random_state= 1004)

SVM 모델 학습

In [ ]:
x_train_men.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38252 entries, 17613 to 48130
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   SEX          38252 non-null  category
 1   AGE          38252 non-null  category
 2   HEIGHT       38252 non-null  category
 3   WEIGHT       38252 non-null  category
 4   BP_HIGH      38252 non-null  float64 
 5   BP_LWST      38252 non-null  float64 
 6   BLDS         38252 non-null  float64 
 7   HMG          38252 non-null  float64 
 8   CREATININE   38252 non-null  float64 
 9   SGOT_AST     38252 non-null  float64 
 10  SGOT_ALT     38252 non-null  float64 
 11  GAMMA_GTP    38252 non-null  float64 
 12  DRK_YN       38252 non-null  category
 13  SIGHT_LEFT   38252 non-null  category
 14  SIGHT_RIGHT  38252 non-null  category
 15  OLIG_PROTE   38252 non-null  category
dtypes: category(8), float64(8)
memory usage: 2.9 MB


In [ ]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16394 entries, 7860 to 25378
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   SEX          16394 non-null  category
 1   AGE          16394 non-null  category
 2   HEIGHT       16394 non-null  category
 3   WEIGHT       16394 non-null  category
 4   WAIST        16394 non-null  float64 
 5   BP_HIGH      16394 non-null  float64 
 6   BP_LWST      16394 non-null  float64 
 7   BLDS         16394 non-null  float64 
 8   HMG          16394 non-null  float64 
 9   CREATININE   16394 non-null  float64 
 10  SGOT_AST     16394 non-null  float64 
 11  SGOT_ALT     16394 non-null  float64 
 12  GAMMA_GTP    16394 non-null  float64 
 13  DRK_YN       16394 non-null  category
 14  SIGHT_LEFT   16394 non-null  category
 15  SIGHT_RIGHT  16394 non-null  category
 16  OLIG_PROTE   16394 non-null  category
dtypes: category(8), float64(9)
memory usage: 1.4 MB


In [ ]:
#남자 거

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x_train_men)
x_train_men = scaler.transform(x_train_men)
x_test_men = scaler.transform(x_test_men)

In [ ]:
scores = []
for thisC in [*range(1,10)]:
    svc=SVC(kernel='linear',C=thisC)
    model=svc.fit(x_train_men,y_train_men)
    scoreTrain=model.score(x_train_men,y_train_men)
    scoreTest=model.score(x_test_men,y_test_men)
    y_predicted = model.predict(x_test_men)
    print("선형 SVM : C:{}, training score:{:2f}, test score:{:2f}".format
          (thisC,scoreTrain, scoreTest))
    print("Classification report for %s" % model)
    print(metrics.classification_report(y_test_men, y_predicted))

    scores.append([scoreTrain, scoreTest])

선형 SVM : C:1, training score:0.613563, test score:0.617177
Classification report for SVC(C=1, class_weight={}, kernel='linear')
              precision    recall  f1-score   support

           0       0.69      0.32      0.44      7604
           1       0.60      0.87      0.71      8790

    accuracy                           0.62     16394
   macro avg       0.64      0.60      0.57     16394
weighted avg       0.64      0.62      0.58     16394

선형 SVM : C:2, training score:0.613563, test score:0.617177
Classification report for SVC(C=2, class_weight={}, kernel='linear')
              precision    recall  f1-score   support

           0       0.69      0.32      0.44      7604
           1       0.60      0.87      0.71      8790

    accuracy                           0.62     16394
   macro avg       0.64      0.60      0.57     16394
weighted avg       0.64      0.62      0.58     16394

선형 SVM : C:3, training score:0.613563, test score:0.617177
Classification report for SVC(C

In [ ]:
men_test=test.drop(['id'],axis=1)
test.head()
testmat=np.array(test)
test=scaler.fit_transform(test)
#test(DF -> array)
model=SVC(kernel='rbf', C=5, gamma=.1).fit(xtrain, ytrain)

In [ ]:
prediction=model.predict(test)
pred=pd.DataFrame(prediction)
pred

In [ ]:
#여자 거

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x_train_women)
x_train_women = scaler.transform(x_train_women)
x_test_women = scaler.transform(x_test_women)

In [ ]:
scores = []
for thisC in [*range(1,10)]:
    svc=SVC(kernel='linear',C=thisC)
    model=svc.fit(x_train_women,y_train_women)
    scoreTrain=model.score(x_train_women,y_train_women)
    scoreTest=model.score(x_test_women,y_test_women)
    y_predicted = model.predict(x_test_women)
    print("선형 SVM : C:{}, training score:{:2f}, test score:{:2f}".format
          (thisC,scoreTrain, scoreTest))
    print("Classification report for %s" % model)
    print(metrics.classification_report(y_test_women, y_predicted))

    scores.append([scoreTrain, scoreTest])

선형 SVM : C:1, training score:0.963367, test score:0.961784
Classification report for SVC(C=1, class_weight={}, kernel='linear')
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     13087
           1       0.00      0.00      0.00       520

    accuracy                           0.96     13607
   macro avg       0.48      0.50      0.49     13607
weighted avg       0.93      0.96      0.94     13607



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


선형 SVM : C:2, training score:0.963367, test score:0.961784
Classification report for SVC(C=2, class_weight={}, kernel='linear')
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     13087
           1       0.00      0.00      0.00       520

    accuracy                           0.96     13607
   macro avg       0.48      0.50      0.49     13607
weighted avg       0.93      0.96      0.94     13607



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


선형 SVM : C:3, training score:0.963367, test score:0.961784
Classification report for SVC(C=3, class_weight={}, kernel='linear')
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     13087
           1       0.00      0.00      0.00       520

    accuracy                           0.96     13607
   macro avg       0.48      0.50      0.49     13607
weighted avg       0.93      0.96      0.94     13607



...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


선형 SVM : C:8, training score:0.963367, test score:0.961784
Classification report for SVC(C=8, class_weight={}, kernel='linear')
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     13087
           1       0.00      0.00      0.00       520

    accuracy                           0.96     13607
   macro avg       0.48      0.50      0.49     13607
weighted avg       0.93      0.96      0.94     13607



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


선형 SVM : C:9, training score:0.963367, test score:0.961784
Classification report for SVC(C=9, class_weight={}, kernel='linear')
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     13087
           1       0.00      0.00      0.00       520

    accuracy                           0.96     13607
   macro avg       0.48      0.50      0.49     13607
weighted avg       0.93      0.96      0.94     13607



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test = pd.read_csv('test.csv')